In [ ]:
# default_exp visualizations

In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np

def plot_pvals(result_df):
    pvals = result_df["peptide_pval"].to_list()
    plt.hist(pvals,99,cumulative=True,density=True, histtype='step')
    x = np.linspace(0,1,100)
    plt.plot(x, x)
    plt.show()

In [ ]:
#export
from scipy.stats import norm
import matplotlib.pyplot as plt

def plot_bgdist(bgdist):
    fc2counts_rescaled = tranform_fc2count_to_fc_space(bgdist.fc2counts, bgdist.cumulative[-1],1/100.0)

    plt.bar(list(fc2counts_rescaled.keys()), fc2counts_rescaled.values(),width=0.01,color='g')
    axes2 = plt.twinx()
    x = np.linspace(-4, 4, 1000)
    axes2.plot(x, norm.pdf(x, 0, bgdist.SD)/1.15)
    axes2.set_ylim(0.0, 0.4)
    plt.show()

def tranform_fc2count_to_fc_space(fc2counts, num_fcs, rescale_factor):
    fc2counts_fcscales = {}
    for fc, count in fc2counts.items():
        fc2counts_fcscales[fc*rescale_factor] = count/num_fcs

    return fc2counts_fcscales

In [ ]:
#export
import matplotlib.pyplot as plt
from scipy import stats

def plot_betweencond_fcs(df_c1_normed, df_c2_normed, get_median):
    both_idx = df_c1_normed.index.intersection(df_c2_normed.index)
    df1 = df_c1_normed.loc[both_idx]
    df2 = df_c2_normed.loc[both_idx]
    if get_median:
        df1 = df1.median(axis = 1, skipna = True).to_frame()
        df2 = df2.median(axis = 1, skipna = True).to_frame()
    for col1 in df1.columns:
        for col2 in df2.columns:
            diff_fcs = df1[col1].to_numpy() - df2[col2].to_numpy()
            mode = stats.mode(diff_fcs, nan_policy='omit')[0][0]
            median = np.nanmedian(diff_fcs)
            #plt.axvline(mode, color = 'blue')
            plt.axvline(median, color = 'red')
            plt.hist(diff_fcs,99,density=True, histtype='step')
    plt.show()

In [ ]:
from scipy import stats
mode = stats.mode([1,3,4,5,6,7,8,4,3,2,46,1,1,1])[0]
print(mode)

[1]


In [ ]:
#export
import matplotlib.pyplot as plt
def scatter_df_columns(merged_df, log_axes = False):
    col = (0.2, 0.4, 0.6, 0.1)
    ref_columns = list(filter(lambda x : "_ref" in x, merged_df.columns.to_list())) #filter the reference columns from the merged df

    for ref in ref_columns:
        compare = ref.replace("_ref", "")
        ax_p = merged_df.plot.scatter(x=ref,y=compare, color = col)
        corr = merged_df[ref].corr(merged_df[compare])
        plt.title(f"{ref} vs. {compare} corr {corr}")
        x = np.linspace(0,merged_df[ref].max(),100)
        plt.plot(x, x)
        if log_axes:
            plt.xscale('log')
            plt.yscale('log')
        plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
def plot_cumhist_dfcols(merged_df):
    col = (0.2, 0.4, 0.6, 0.4)
    ref_columns = list(filter(lambda x : "_ref" in x, merged_df.columns.to_list())) #filter the reference columns from the merged df

    for ref in ref_columns:
        compare = ref.replace("_ref", "")
        plt.hist(merged_df[ref], 100, density=True, histtype='step', label='reference')
        plt.hist(merged_df[compare], 100, density=True, histtype='step',label='compare')
        corr = merged_df[ref].corr(merged_df[compare])
        plt.title(f"{ref} vs. {compare} corr {corr}")
        plt.show()

In [ ]:
#export
from matplotlib_venn import venn2
from matplotlib import pyplot as plt
def compare_peptid_protein_overlaps(protein_ref, protein_comp, peptide_ref, peptide_comp):
    protIDs_ref = set(protein_ref["protein"].to_list())
    protIDs_comp = set(protein_comp["protein"].to_list())
    venn2([protIDs_ref, protIDs_comp], ('protIDs_ref', 'protIDs_comp'))
    plt.show()
    pepIDs_ref = set(peptide_ref["peptide"].to_list())
    pepIDs_comp = set(peptide_comp["peptide"].to_list())
    venn2([pepIDs_ref, pepIDs_comp], ('pepIDs_ref', 'pepIDs_comp'))
    plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
import seaborn as sns

def plot_fold_change(df, key1, key2):
    to_plot = df.copy()
    to_plot[f'Ratio ({key1}/{key2})'] = np.log2(to_plot[key1] / to_plot[key2])
    to_plot[f'Inten_{key1}'] = np.log10(to_plot[key1])

    species = 'Human'
    val = to_plot.loc[to_plot['species']==species, f'Ratio ({key1}/{key2})'].values
    val = val[~np.isnan(val)&~np.isinf(val)&~np.isneginf(val)]
    print(f'Species={species}, n={len(val)}, median={np.median(val)}, dev={np.std(val)}')
    species='Ecoli'
    val = to_plot.loc[to_plot['species']==species, f'Ratio ({key1}/{key2})'].values
    val = val[~np.isnan(val)&~np.isinf(val)&~np.isneginf(val)]
    print(f'species={species}, n={len(val)}, median={np.median(val)}, dev={np.std(val)}')

    plt.figure(figsize=(7,7))
    ax = sns.scatterplot(x=f'Ratio ({key1}/{key2})', y=f'Inten_{key1}', hue="species", data=to_plot, alpha=0.5)
    plt.title('Fold Change')
    plt.xlim([-4.5, 6.5])
    #plt.ylim([6,11.5])
    plt.show()